In [1]:
import pandas as pd

from functions import join_stocks_crypto, run_clustering_model

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
kshape_results = pd.read_csv('new_balance_silhouette/kshape.csv')
results = kshape_results

In [3]:
from sklearn.preprocessing import MinMaxScaler
results[['silhouette_norm']] = MinMaxScaler().fit_transform(results[['silhouette_score']])
results['delta_norm'] = 1 - MinMaxScaler().fit_transform(results[['min_max_delta']])
results['total_score'] = (results['silhouette_norm'] + results['delta_norm']) / 2

best_configs = results.loc[results.groupby('method')['total_score'].idxmax()]


In [4]:
import numpy as np

df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data_new.csv', index_col='timestamp')
joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left')

cryptos_list = list(cryptos_df.columns)


log_returns = np.log(df_all_stocks / df_all_stocks.shift(1)).dropna()
daily_volatility = log_returns.std()
annual_volatility = daily_volatility * np.sqrt(252)

annual_volatility.sort_values().tail(20)

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


PRCT      0.692412
TTD       0.708151
3YU.F     0.747658
MDB       0.755362
SITM      0.765612
RNA       0.797502
AXSM      0.814172
DJT       0.901152
TBA.F     0.916075
ECX       0.923325
RYTM      0.941110
OL5.F     1.315230
SMMT      1.362448
4AH1.F    1.370364
2B5.F     1.385327
PJXC.F    1.894676
LPS1.F    3.012170
WK0.F     3.041260
0DH.F     4.547030
LFU2.F    4.970240
dtype: float64

In [5]:
df_all_stocks = df_all_stocks.drop(columns=['OL5.F', 'SMMT', '4AH1.F', '2B5.F', 'PJXC.F', 'LPS1.F', 'WK0.F', '0DH.F', 'LFU2.F'])

df_all_stocks.to_csv('stocks_data_filtered_volatility.csv')

tickers = list(df_all_stocks.columns)

In [6]:
from functions import get_tickers_stocks, get_close_prices, double_listed_stocks
import pandas as pd




#GET THE STOCKS

# us_exchanges = ['NMS', 'NYQ', 'NGM']
# eu_exchanges = ['PAR', 'FRA', 'LSE', 'AMS']
# asia_exchanges = ['SHH', 'JPX', 'HKG']

# selected_exchanges = us_exchanges + eu_exchanges + asia_exchanges

# full_selected_stocks = {}
# df_all_stocks = pd.DataFrame()
# for exchange in selected_exchanges:
#     print(f'Extracting from {exchange}')
#     exchanges = [exchange]
#     selected_stocks_dict, ticker_list = get_tickers_stocks(50000, exchanges, 20)

#     full_selected_stocks.update(selected_stocks_dict)

if len(tickers) > 0: 
    df = get_close_prices(tickers, period = 1, start = '2024-01-01')
    #df_all_stocks = pd.concat([df_all_stocks, df], axis=1)

# doubly_listed_tickers = double_listed_stocks(df)

# for ticker_to_drop in doubly_listed_tickers:
#     try:
#         df_all_stocks = df_all_stocks.drop(columns=[ticker_to_drop])
#     except:
#         pass

df_all_stocks = df.ffill() #ffill again after concatenating the tickers



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  170 of 170 completed


In [8]:
df_all_stocks.to_csv('stocks_data_out_sample_2024_foltered_volatility.csv')

In [ ]:
def run_simulation(portfolio_dict:dict, returns_for_portfolio:pd.DataFrame, n_sims=100, t=100, distribution_model='multivar_norm', 
                   plot=False, initialPortfolio=100, winsorize=False, winsorize_limits=(0.01, 0.01)):
    returns_for_portfolio = returns_for_portfolio[list(portfolio_dict.keys())]
    
    if winsorize:

        winsorized_returns = returns_for_portfolio.copy()
        
        for col in winsorized_returns.columns:
            winsorized_returns[col] = mstats.winsorize(winsorized_returns[col], limits=winsorize_limits)
        

        returns_for_portfolio = winsorized_returns


    mean_returns = returns_for_portfolio.mean()
    cov_matrix = returns_for_portfolio.cov()

    weights = [v for _, v in portfolio_dict.items()]

    meanM = np.tile(mean_returns, (t, 1))  # Shape: (T, n_assets)

    portfolio_sims = np.zeros((t, n_sims))

    L = np.linalg.cholesky(cov_matrix)  # Cholesky decomposition

    for sim in range(n_sims):

        if distribution_model == 'bootstrap':
            sampled_returns = returns_for_portfolio.bfill().sample(n=t, replace=True).values
            portfolio_returns = sampled_returns @ weights
        elif distribution_model in ['multivar_norm', 'multivar_t']:

            if distribution_model == 'multivar_norm':
                Z = np.random.normal(size=(t, len(portfolio_dict)))  # Shape: (T, n_assets)
                daily_returns = meanM + Z @ L.T  # Shape: (T, n_assets)
            elif distribution_model == 'multivar_t':
                df = 25  # degrees of freedom
                Z = np.random.normal(size=(t, len(portfolio_dict)))
                chi2 = np.random.chisquare(df, size=(t, 1))
                Z_t = Z / np.sqrt(chi2 / df)  # now Z_t has t-distributed marginals

                daily_returns = meanM + Z_t @ L.T

            portfolio_returns = daily_returns @ weights  # Shape: (T,)
        
        else:
            break
            
        
        
        
        portfolio_sims[:, sim] = np.cumprod(1 + portfolio_returns) * initialPortfolio


    if plot:
        plt.plot(portfolio_sims)
        plt.title("Monte Carlo Simulated Portfolio Paths")
        plt.xlabel("Days")
        plt.ylabel("Portfolio Value")
        plt.show()

    return portfolio_sims

def out_of_sample_analysis(portfolio, price_data, start_date, period=126, 
                           calculate_metrics=True, plot_results=False, risk_free_rate=0.02, use_log_returns=True):
    """
    Perform traditional out-of-sample analysis for a portfolio over a specified forward period.
    
    Parameters:
    -----------
    portfolio : dict
        Dictionary with tickers as keys and weights as values
    price_data : pandas.DataFrame
        DataFrame with dates as index and tickers as columns containing PRICES (not returns)
    start_date : str or datetime
        Date marking the beginning of out-of-sample period
    period : int
        Number of days for the forward period to analyze (default 126 ~= 6 months of trading days)
    calculate_metrics : bool
        Whether to calculate and return performance metrics
    plot_results : bool
        Whether to generate and return performance plots
    risk_free_rate : float
        Annual risk-free rate used for Sharpe ratio calculation
    
    Returns:
    --------
    results : dict
        Dictionary containing performance metrics and return series for the period
    """
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)
    
    results = {}
    tickers = list(portfolio.keys())
    weights = np.array([portfolio[ticker] for ticker in tickers])
    
    missing_tickers = [ticker for ticker in tickers if ticker not in price_data.columns]
    if missing_tickers:
        raise ValueError(f"The following tickers are missing from price_data: {missing_tickers}")
    
    price_subset = price_data[tickers].copy()
    price_subset = price_subset.sort_index()
    oos_data = price_subset[price_subset.index >= start_date]
    
    if oos_data.empty:
        raise ValueError(f"No data available after start_date: {start_date}")
    
    end_date = start_date + timedelta(days=period)
    
    # Folter data for this period
    period_mask = (oos_data.index <= end_date)
    period_data = oos_data.loc[period_mask]
    


    returns = period_data.dropna()
    original_returns = returns.copy()

    if use_log_returns:
        invalid_mask = returns <= -1.0
        if invalid_mask.any().any():
            invalid_stocks = returns.columns[invalid_mask.any()]

            log_compatible_returns = returns.copy()

            log_compatible_returns[invalid_mask] = -0.99
            
            log_returns = np.log(1 + log_compatible_returns)
        else:
            log_returns = np.log(1 + returns)
            
        portfolio_log_returns = log_returns.dot(weights)
        
        # Convert back to simple returns for statistics
        portfolio_returns = np.exp(portfolio_log_returns) - 1
        
        # Calculate cumulative returns properly from log returns
        cumulative_returns = np.exp(portfolio_log_returns.cumsum()) - 1
    else:
        # Simple returns method
        portfolio_returns = returns.dot(weights)
        cumulative_returns = (1 + portfolio_returns).cumprod() - 1






    results = {
        'returns': portfolio_returns,
        'cumulative_returns': cumulative_returns
    }
    
    if calculate_metrics:
        total_return = cumulative_returns.iloc[-1] if len(cumulative_returns) > 0 else 0
        trading_days = len(portfolio_returns)
        
        # Annualization factor
        annual_factor = 252 / trading_days
        annualized_return = (1 + total_return) ** annual_factor - 1
        original_portfolio_returns = original_returns.dot(weights)
        volatility_annual = original_portfolio_returns.std() * np.sqrt(252)
        sharpe_ratio = (annualized_return - risk_free_rate) / volatility_annual if volatility_annual > 0 else 0
        non_annual_sharpe = (total_return - risk_free_rate) / portfolio_returns.std()
    
        
        if len(portfolio_returns) > 2:
            var_95 = abs(portfolio_returns.quantile(0.05))
        
        results.update({
            'mean_period_return': total_return,
            'annualised_return': annualized_return,
            'sharpe_annualized': sharpe_ratio,
            'sharpe_period': non_annual_sharpe,
            'var_95': var_95,
        })
    
    if plot_results and len(cumulative_returns) > 0:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot cumulative returns
        cumulative_returns.plot(ax=ax1, title=f'{period}-Day Cumulative Returns')
        ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        ax1.set_ylabel('Cumulative Return')
        ax1.grid(True)
        
        # Plot drawdowns
        drawdowns.plot(ax=ax2, title=f'{period}-Day Drawdowns', color='red')
        ax2.set_ylabel('Drawdown')
        ax2.set_ylim(bottom=min(drawdowns.min() * 1.1, -0.05), top=0.01)
        ax2.grid(True)
        
        plt.tight_layout()
        results['plot'] = fig
    
    return results


def simulate_evaluate_portfolio_subset(portfolios_subset:dict, return_df, n_sims=100, t=100, distribution_model='out_sample_direct', rf_annual=0.04, seed=30,
                                       winsorize=False, winsorize_limits=(0.01, 0.01)):
    simulations_results_dict = dict()
    subset_statistics_df = pd.DataFrame()

    random.seed(seed)
    #if distribution_model=='out_sample_direct':
            #print("Don't forget to put 2024 data in return_df!!!!")

    for i, portfolio_dict in portfolios_subset.items():
        returns_portfolio = return_df[list(portfolio_dict.keys())]

        #Making sure weights sum up to 1
        total = sum(portfolio_dict.values())
        portfolio_dict = {k: v / total for k, v in portfolio_dict.items()}

        



        if distribution_model=='out_sample_direct':
            res = out_of_sample_analysis(portfolio_dict, return_df, '2024-01-01', period=t, calculate_metrics=True, risk_free_rate=rf_annual, plot_results=False, use_log_returns=False)
            mean_annual_return = res['annualised_return']
            mean_return = res['mean_period_return']
            sharpe_annual = res['sharpe_annualized']
            VaR_final = res['var_95']
            sharpe_period = res['sharpe_period']

            stat_results = pd.DataFrame({'annualised_return': [mean_annual_return],
                                     'mean_period_return': [mean_return],
                                     'sharpe_annualized': [sharpe_annual],
                                     'sharpe_period': [sharpe_period],
                                     'VaR': [VaR_final],
                                     })
            
            subset_statistics_df = pd.concat([subset_statistics_df, stat_results])

            continue
        
        else:
            portfolio_sims = run_simulation(portfolio_dict, returns_portfolio, n_sims=n_sims, t=t, distribution_model=distribution_model, plot=False, winsorize=winsorize, winsorize_limits=winsorize_limits)



            simulations_results_dict[i] = portfolio_sims


            #CALCULATE STATISTICS PER PORTFOLIO:
            daily_returns = (portfolio_sims[1:, :] - portfolio_sims[:-1, :]) / portfolio_sims[:-1, :] #DO WE WANT TO ADD FIRST ROW OF 100 VALUES SO THAT WE HAVE A FULL THING?
            #average cumulative return 
            cumulative_returns_per_simulation = calculate_cumulative_returns(daily_returns)
            #mean_cumulative_return_for_portfolio = np.mean(cumulative_returns_per_simulation)
            #average daily return
            mean_daily_return_for_portfolio = np.mean(daily_returns)
            #STD
            std_daily_return = np.std(daily_returns) #return variability per day

            #annualised_return = (1 + mean_cumulative_return_for_portfolio)**(252/t) - 1
            #annualised_volatility = std_cumulative_return * np.sqrt(252/t)




            final_portfolio_values = portfolio_sims[-1, :]  # Final values across all simulations
            initial_value = 100  # Your initial portfolio value
            holding_period_years = t / 252  # Convert days to years

            # Calculate annualized returns 
            return_for_period = (final_portfolio_values - initial_value) / initial_value
            annualised_return = (final_portfolio_values / initial_value) ** (1 / holding_period_years) - 1
            #annualised_return = (1 + return_for_period) ** (1/holding_period_years) - 1
            mean_return = np.mean(return_for_period)
            mean_annual_return = np.mean(annualised_return)




            #sharpe ratio
            rf_daily = rf_annual / 252
            #rf_cumulative = (1 + rf_annual) ** (daily_returns.shape[0] / 252) - 1
            sharpe_daily = (mean_daily_return_for_portfolio - rf_daily) / std_daily_return
            #sharpe_cumulative = (mean_cumulative_return_for_portfolio - rf_cumulative)  / std_cumulative_return
            sharpe_annual = sharpe_daily * np.sqrt(252)
            #sharpe_period = 
            #sharpe_cumulative_annual = sharpe_cumulative * np.sqrt(252)

            #Sortino ratio
            # downside_returns = np.minimum(0, daily_returns - rf_daily)

            # with np.errstate(over='ignore'):  # Suppress the warning
            #     squared_returns = np.square(downside_returns, dtype=np.float64)
            #     downside_std = np.sqrt(np.mean(squared_returns))

            # if np.isclose(downside_std, 0) or np.isnan(downside_std):
            #     sortino_ratio = np.nan  # or set to a default value
            # else:
            #     sortino_ratio = (mean_daily_return_for_portfolio - rf_daily) / downside_std

            #sortino_annual = sortino_ratio * np.sqrt(252)

            #VaR:
            last_period_returns = portfolio_sims[-1:]
            initial_portfolio_value = 100
            portfolio_returns = (last_period_returns - initial_portfolio_value) / initial_portfolio_value
            VaR = np.percentile(portfolio_returns, 5)
            VaR_final = abs(VaR) * initial_portfolio_value
            #CVaR
            worst_losses = portfolio_returns[portfolio_returns <= VaR]
            CVaR_final = abs(worst_losses.mean()) * initial_portfolio_value

            VaR_final, CVaR_final = calculate_var_cvar(daily_returns, 
                                                 confidence_level=0.05, 
                                                 initial_value=initial_portfolio_value)


            stat_results = pd.DataFrame({'annualised_return': [mean_annual_return],
                                     'mean_period_return': [mean_return],
                                     #'annualised_volatility': [annualised_volatility],
                                     #'std_daily_return': [std_daily_return],
                                     #'sharpe_daily': [sharpe_daily],
                                     #'sharpe_cumulative': [sharpe_cumulative],
                                     'sharpe_annualized': [sharpe_annual],
                                     'VaR': [VaR_final],
                                     #'CVaR': [CVaR_final],
                                     #'sortino': [sortino_ratio],
                                     #'sortino_annualized': [sortino_annual]
                                     })

            subset_statistics_df = pd.concat([subset_statistics_df, stat_results])

    subset_statistics_df = subset_statistics_df.reset_index(drop=True)

    #RUN THE NORMALITY TEST
    results_normality_test = {}
    for col in subset_statistics_df.columns:
        stat, p_value = normaltest(subset_statistics_df[col])
        results_normality_test[col] = {'statistic': stat, 'p_value': p_value}

    normality_results_df = pd.DataFrame(results_normality_test).T
    normality_results_df['normal'] = normality_results_df['p_value'] > 0.05


    return simulations_results_dict, subset_statistics_df, normality_results_df
